# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: No module named 'pymorphy2'

In [100]:
import pandas as pd
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import numpy as np
import random
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [13]:
pred = pd.read_csv('preprocessed_descriptions.csv'); pred

,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [15]:
b = [word_tokenize(str(i)) for i in pred['description']]; b

[['an',
  'original',
  'recipe',
  'created',
  'by',
  'chef',
  'scott',
  'meskan',
  'georges',
  'at',
  'the',
  'cove',
  'we',
  'enjoyed',
  'this',
  'when',
  'we',
  'visited',
  'this',
  'restaurant',
  'in',
  'la',
  'jolla',
  'california',
  'this',
  'recipe',
  'is',
  'requested',
  'so',
  'often',
  'they',
  'have',
  'it',
  'printed',
  'and',
  'ready',
  'at',
  'the',
  'hostess',
  'stand',
  'its',
  'unbeatable',
  'at',
  'the',
  'restaurant',
  'but',
  'i',
  'do',
  'a',
  'pretty',
  'good',
  'job',
  'at',
  'home',
  'too',
  'if',
  'i',
  'do',
  'say',
  'so',
  'myself'],
 ['my',
  'children',
  'and',
  'their',
  'friends',
  'ask',
  'for',
  'my',
  'homemade',
  'popsicles',
  'morning',
  'noon',
  'and',
  'night',
  'i',
  'never',
  'turn',
  'them',
  'down',
  'who',
  'am',
  'i',
  'to',
  'tell',
  'them',
  'that',
  'they',
  'are',
  'good',
  'for',
  'them',
  'for',
  'variety',
  'i',
  'substitute',
  'different',
  'f

In [28]:
b1 = []
[b1.append(str(j)) for i in b for j in i]; b1
b2 = np.unique(b1); b2

array(['0', '00', '00mg', ..., 'zwtii', 'zwtlaos', 'zzar'], dtype='<U146')

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [40]:
b4 = []
for i in range(5):
    b4.append(random.choices(b2, k = 2))

In [43]:
b4[0][0]

'nzlambcom'

In [46]:
b6 = []
for i in b4:
    b6.append(edit_distance(i[0], i[1]))

In [47]:
b6

[8, 6, 6, 7, 9]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [86]:
def nearwords(word, n, listt):
    b = {}
    for i in listt:
        b.update({i: edit_distance(word, i)})
    sorted_b = dict(sorted(b.items(), key=lambda x: x[1]))
    return list(sorted_b.keys())[1:n]

In [87]:
nearwords('hello', 6, b2)

['hell', 'jello', 'mello', 'bell', 'bella']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [89]:
stem = SnowballStemmer('english')
lem = WordNetLemmatizer()

In [92]:
lem1 = [lem.lemmatize(i) for i in b2]
stem1 = [stem.stem(i) for i in b2]

In [94]:
data = pd.DataFrame({'word': b2, 'stemmed_word': stem1, 'normalized_word': lem1})
data = data.set_index('word'); data


,stemmed_word,normalized_word
word,,
0,0,0
00,00,00
00mg,00mg,00mg
01,01,01
01000,01000,01000
...,...,...
zwt8great,zwt8great,zwt8great
zwt9,zwt9,zwt9
zwtii,zwtii,zwtii


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [101]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [121]:
len(b1)

1072496

In [122]:
b7 = [i for i in b1 if i not in stopwords.words('english')]

In [123]:
b7

['original',
 'recipe',
 'created',
 'chef',
 'scott',
 'meskan',
 'georges',
 'cove',
 'enjoyed',
 'visited',
 'restaurant',
 'la',
 'jolla',
 'california',
 'recipe',
 'requested',
 'often',
 'printed',
 'ready',
 'hostess',
 'stand',
 'unbeatable',
 'restaurant',
 'pretty',
 'good',
 'job',
 'home',
 'say',
 'children',
 'friends',
 'ask',
 'homemade',
 'popsicles',
 'morning',
 'noon',
 'night',
 'never',
 'turn',
 'tell',
 'good',
 'variety',
 'substitute',
 'different',
 'flavours',
 'frozen',
 'juice',
 'grape',
 'fruit',
 'punch',
 'tropical',
 'etc',
 'go',
 'surprised',
 'even',
 'sisterinlaw',
 'made',
 'us',
 'family',
 'get',
 'together',
 'delicious',
 'little',
 'messy',
 'make',
 'worth',
 'effort',
 'helper',
 'make',
 'think',
 'fondue',
 'romantic',
 'casual',
 'dinner',
 'wonderful',
 'theatre',
 'snack',
 'served',
 'robust',
 'red',
 'wine',
 'dinner',
 'serve',
 'rice',
 'small',
 'salad',
 'almond',
 'rice',
 'pilaf',
 'great',
 'accompaniment',
 'recipe',
 'pos

In [124]:
(len(b1) - len(b7))/(len(b1))

0.45682781101281494

In [125]:
b8 = [i for i in b1 if i in stopwords.words('english')]; b8

['an',
 'by',
 'at',
 'the',
 'we',
 'this',
 'when',
 'we',
 'this',
 'in',
 'this',
 'is',
 'so',
 'they',
 'have',
 'it',
 'and',
 'at',
 'the',
 'its',
 'at',
 'the',
 'but',
 'i',
 'do',
 'a',
 'at',
 'too',
 'if',
 'i',
 'do',
 'so',
 'myself',
 'my',
 'and',
 'their',
 'for',
 'my',
 'and',
 'i',
 'them',
 'down',
 'who',
 'am',
 'i',
 'to',
 'them',
 'that',
 'they',
 'are',
 'for',
 'them',
 'for',
 'i',
 'of',
 'these',
 'were',
 'so',
 'it',
 'me',
 'my',
 'these',
 'for',
 'at',
 'a',
 'they',
 'are',
 'a',
 'to',
 'but',
 'the',
 'have',
 'a',
 'and',
 'an',
 'i',
 'a',
 'is',
 'a',
 'very',
 'or',
 'for',
 'after',
 'the',
 'with',
 'a',
 'for',
 'with',
 'a',
 'is',
 'a',
 'to',
 'the',
 'you',
 'your',
 'i',
 'do',
 'this',
 'by',
 'it',
 'to',
 'on',
 'the',
 'and',
 'then',
 'it',
 'to',
 'your',
 'i',
 'only',
 'a',
 'have',
 'at',
 'if',
 'you',
 'to',
 'just',
 'the',
 'to',
 'there',
 'will',
 'be',
 'from',
 'my',
 'has',
 'been',
 'these',
 'are',
 'a',
 'our',


In [126]:
b9 = np.unique(b8, return_counts = True)

In [129]:
data1 = pd.DataFrame({'word': b9[0], 'popularity': b9[1]})

In [132]:
data1 = data1.sort_values(by='popularity', ascending=False)
data1.head(10)

,word,popularity
98,the,40210
0,a,34994
9,and,30279
107,this,27048
52,i,25111
110,to,23499
56,is,20290
57,it,19863
74,of,18372
35,for,15988


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).